In [15]:
# import necessary 

import numpy as np     # linear algebra library
import pandas as pd    
import tensorflow as tf

from random import randint

import matplotlib.pyplot as plt    # data visualization
plt.style.use('seaborn-white')
import seaborn as sns              # data visualization
sns.set_style("white")

from sklearn.model_selection import train_test_split        # function used for splitting the data 
                                                            # in training and validation, with the
                                                            # same distribution (based on salt coverage)

from skimage.transform import resize         

# keras specifics

from keras.preprocessing.image import load_img
from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation, Add, Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout
from keras import backend as K
# import BatchNormalization
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras.layers.core import Lambda

from tqdm import tqdm_notebook
from tensorflow.compat.v1.metrics import mean_iou

In [7]:
location = "C://Users//oana.sabadas//PycharmProjects//TGS//tgs-salt-identification-challenge//Numpy//splitted//128"   
#location of the files to be read

In [8]:
y_train = np.load("{}//y_train.npy".format(location),allow_pickle=True)
x_train = np.load("{}//x_train.npy".format(location),allow_pickle=True)


y_valid = np.load("{}//y_valid.npy".format(location),allow_pickle=True)
x_valid = np.load("{}//x_valid.npy".format(location),allow_pickle=True)


ids_train = np.load("{}//ids_train.npy".format(location),allow_pickle=True)
ids_valid = np.load("{}//ids_valid.npy".format(location),allow_pickle=True)


cov_train = np.load("{}//cov_train.npy".format(location),allow_pickle=True)
cov_valid = np.load("{}//cov_test.npy".format(location),allow_pickle=True)


depth_train = np.load("{}//depth_train.npy".format(location),allow_pickle=True)
depth_valid = np.load("{}//depth_test.npy".format(location),allow_pickle=True)


In [9]:
x_train.shape     # check te shape to see if everything is alright

(3200, 128, 128, 1)

In [10]:

def iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [17]:
def BatchActivation(x):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

In [18]:
# Build U-Net model
inputs = Input((128, 128, 1))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(8, (3, 3), padding='same') (s)
c1 = BatchActivation(c1)
c1 = Conv2D(8, (3, 3), padding='same') (c1)
c1 = BatchActivation(c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(16, (3, 3), padding='same') (p1)
c2 = BatchActivation(c2)
c2 = Conv2D(16, (3, 3), padding='same') (c2)
c2 = BatchActivation(c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(32, (3, 3), padding='same') (p2)
c3 = BatchActivation(c3)
c3 = Conv2D(32, (3, 3), padding='same') (c3)
c3 = BatchActivation(c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(64, (3, 3), padding='same') (p3)
c4 = BatchActivation(c4)
c4 = Conv2D(64, (3, 3), padding='same') (c4)
c4 = BatchActivation(c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(128, (3, 3), padding='same') (p4)
c5 = BatchActivation(c5)
c5 = Conv2D(128, (3, 3), padding='same') (c5)
c5 = BatchActivation(c5)

u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(64, (3, 3), padding='same') (u6)
c6 = BatchActivation(c6)
c6 = Conv2D(64, (3, 3), padding='same') (c6)
c6 = BatchActivation(c6)

u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(32, (3, 3), padding='same') (u7)
c7 = BatchActivation(c7)
c7 = Conv2D(32, (3, 3), padding='same') (c7)
c7 = BatchActivation(c7)

u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(16, (3, 3), padding='same') (u8)
c8 = BatchActivation(c8)
c8 = Conv2D(16, (3, 3), padding='same') (c8)
c8 = BatchActivation(c8)

u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(8, (3, 3), padding='same') (u9)
c9 = BatchActivation(c9)
c9 = Conv2D(8, (3, 3), padding='same') (c9)
c9 = BatchActivation(c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

W0809 15:43:14.292227 15680 deprecation_wrapper.py:119] From C:\Users\oana.sabadas\AppData\Local\Continuum\anaconda3\envs\tensorflow_keras\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0809 15:43:14.294239 15680 deprecation_wrapper.py:119] From C:\Users\oana.sabadas\AppData\Local\Continuum\anaconda3\envs\tensorflow_keras\lib\site-packages\keras\backend\tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0809 15:43:14.333216 15680 deprecation_wrapper.py:119] From C:\Users\oana.sabadas\AppData\Local\Continuum\anaconda3\envs\tensorflow_keras\lib\site-packages\keras\backend\tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0809 15:43:14.516096 15680 deprecation_wrapper.py:119] From C:\Users\oana.sabadas\AppData\Local\Continuum\anaconda3\e

In [19]:
model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[iou])
model.summary()

W0809 15:43:19.121247 15680 deprecation_wrapper.py:119] From C:\Users\oana.sabadas\AppData\Local\Continuum\anaconda3\envs\tensorflow_keras\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0809 15:43:19.166219 15680 deprecation.py:323] From C:\Users\oana.sabadas\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0809 15:43:19.192213 15680 deprecation.py:323] From <ipython-input-10-712004aaa345>:4: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0809 15:43:19.297158 15680 deprecation.py:323] From C:\Users\oana.sabadas\AppData\Roaming\Python\Py

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 128, 128, 1)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 128, 128, 8)  80          lambda_2[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 128, 8)  32          conv2d_2[0][0]                   
__________________________________________________________________________________________________
activation

batch_normalization_11 (BatchNo (None, 16, 16, 64)   256         conv2d_12[0][0]                  
__________________________________________________________________________________________________
activation_11 (Activation)      (None, 16, 16, 64)   0           batch_normalization_11[0][0]     
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 16, 16, 64)   36928       activation_11[0][0]              
__________________________________________________________________________________________________
batch_normalization_12 (BatchNo (None, 16, 16, 64)   256         conv2d_13[0][0]                  
__________________________________________________________________________________________________
activation_12 (Activation)      (None, 16, 16, 64)   0           batch_normalization_12[0][0]     
__________________________________________________________________________________________________
conv2d_tra

In [20]:
model.save("models/unet_vanilla_128_adam_binary-crossentropy_iou-metric.h5")